In [4]:
import pandas

In [5]:
def transform_time(a):
    minute = a // 60
    second = a % 60
    
    if minute < 10:
        minute_str = '0' + str(minute)
    else:
        minute_str = str(minute)
    
    if second < 10:
        second_str = '0' + str(second)
    else:
        second_str = str(second)
        
    time = minute_str + ':' + second_str
    return time

In [6]:
# 그여축(아델)
reduce = [5, 2] #[메르 쿨감, 쿨뚝]
cooltime_orgin = 240
cooltime = cooltime_orgin * (1 - reduce[0]/100) - reduce[1] # 실제 쿨타임
nuking = [['정오', 0], ['석양', 135], ['자정', 265], ['1바퀴', 420]] # 바인드 타이밍
title = ['페이즈', '시간', '사용여부', '쿨타임', '남은 스택']

In [7]:
candidate = [1, 4, 7] #여명 자정 때는 무조건 올림 
buff = [1]*9
result = []
count = []
i = 0
digit = len(candidate)
number = 2 ** digit # 따져야하는 경우의 수

#전수검사
while i < number:
    j = i 
    off = []
    shutdown = 0 # 불가능한 상황인지 판단하는 지표
    
    # 2진수 변환
    for _ in range(0, digit):
        a = j % 2
        j = j//2
        off.append(a)
    
    # 케이스설정
    for x in range(0, digit):
        buff[candidate[x]] = off[x]
    
    stack = 2
    now = 0
    time = 0
    total = []
    
    # 케이스 시뮬레이션
    for k in range(0, 9):
        information = [] # 각 페이즈의 정보를 담음
        l = k % 3 # 0이면 정오, 1이면 석양, 2이면 자정
        information.append(nuking[l][0])
        
        #현시점 남은 쿨타임
        if k == 0:
            now += cooltime # 초기값 설정(시작할 때 무조건 사용함)
        else:
            if l == 0:
                duration = nuking[3][1] - nuking[2][1] #정오일 때
            else:
                duration = nuking[l][1] - nuking[l-1][1] #석양, 자정일 때
            
            time += duration

            if now >= duration:
                now -= duration
            else:
                now = now - duration + cooltime
                stack += 1
        
        information.append(time)
        
        if stack == 2:
            now = cooltime #스택이 2일 경우 다시 초기값
            
        information.append(now)
        
        # 버프 사용여부 검사
        if buff[k] == 1:
            if stack > 0:
                stack -= 1
                information.append('O')
            else:
                shutdown += 1
                break # 불가능한 상황일 경우 루프 중단
        else:
            information.append('X')
        
        information.append(stack)
        information[2], information[3] = information[3], information[2]
        total.append(information)
    
    # break를 했으면 데이터 파기, 아니면 저장
    if shutdown == 0:
        result.append(total)
        count.append(buff.count(0))
    else:
        pass
    i += 1

final = []
optimal = min(count)

# 가능한 경우의 수 중 가동률이 제일 높은 케이스 선별
for y in range(0, len(count)):
    if count[y] == optimal:
        final.append(result[y])
    else:
        pass

# 리스트를 데이터프레임으로
for val in final:
    df = pandas.DataFrame(val, columns = title)
    display(df)
        
        
           

,페이즈,시간,사용여부,쿨타임,남은 스택
0,정오,0,O,226.0,1
1,석양,135,X,91.0,1
2,자정,265,O,226.0,1
3,정오,420,O,71.0,0
4,석양,555,X,162.0,1
5,자정,685,O,32.0,0
6,정오,840,O,103.0,0
7,석양,975,X,194.0,1
8,자정,1105,O,64.0,0


[0, 1, 0]